# Моделювання гри в теніс.

Постановка задачі:

Є стандартний матч з настільного тенісу.
Гравці по черзі роблять по 2 подачі, поки рахунок не стане 10-10 або один
їх не набере 11. Після 10-10 починається овертайм.

Овертайм
закінчується, як тільки один із гравців набере перевагу в 2 очки.

Допустимо, у нас є 2 параметри:
- p1 – ймовірність взяття раунду (подачі) першим гравцем на своїй
подачі
- p2 – ймовірність взяття раунду (подачі) першим гравцем на подачі
суперника.

# Завдання

1. Вважаємо, що матч не переходить до овертайму (за рахунку 10-10 гра

закінчується). Потрібно реалізувати алгоритм, який приймає на вхід

p1 та p2, на вихід видає розподіл фінальних рахунків у форматі

{(score1, score2): probability} трьома способами:

А) Оцінка методом Монте Карло.

Б) Прохід можливих наслідків по дереву.

В) Пошуком стаціонарного розподілу, використовуючи матрицю

переходів Марковського процесу.


2. Вважаємо, що стартовий рахунок 10-10. Реалізувати алгоритм оцінки

ймовірності перемоги кожного гравця в овертаймі за умови, що p1=p2,

тими самими способами.


3. Опис порівняльних плюсів та мінусів методів А, Б, В для обох

пунктів.


In [ ]:
import numpy as np
import pandas as pd

# 1 

## Монте-Карло

Розберемося в чому полягає суть метода Монте-Карло.

У нас є деяка випадкова подія, в нашому випадку партія в теніс, ми моделюємо цю випадкову подію і записуємо кількість результатів, які отримали, наприклад, при 1000 моделюваннях ми отримали (10,10) - 100 разів, (11,7) - 60 разів,..., тоді для того щоб знайти ймовірності, з якими деяка подія трапляється потрібно поділити кількість випадків, у яких трапилася ця подія на кількість моделювань : P(ksi = (10,10)) = 100/1000.

Чому так?

Розглянемо випадкову величину X_i, яка буде індикатором деякого результату i-тої гри, наприклад, (10,10), тоді вона має розподіл X_i ={ 1: p, 0:1-p}, тобто ймовірність того, що i-та гра закінчиться з результатом (10,10) - p, а з іншим - 1-p, тоді (X_1 + X_2 +....+ X_n)/n -> E(X_1), при n-> inf, а E(X_1) = p, тобто за законом великих чисел, сума індикаторів деякого результату гри(яка дорівнює кількості випадків, у яких ця подія трапилась), поділити на кількість моделювань прямує до ймовірності отримати цей результат гри.

Отже, якщо вибрати n великим ми отримаємо досить точну оцінку нашої ймовірності.

In [2]:
# Функція для перевірки, чия подача.
def check_first_player_serve(total_score):
         return total_score % 4 == 0 or (total_score-1) % 4 == 0
    

In [3]:
## Monte-Carlo
def get_ith_outcome(p1,p2):
        first = 0
        second = 0
        while first != 11 and second != 11 and not(second == 10 and first == 10):
            random_number = np.random.uniform()
            first_player_serve = check_first_player_serve(first+second)
            if first_player_serve == True and random_number < p1:
                first += 1
            elif first_player_serve == True and random_number > p1:
                second += 1
            elif first_player_serve == False and random_number < p2:
                first += 1
            else:
                second +=1

        return (first,second)
                
def monte_carlo(p1,p2):
    distribution = dict()
    total_trials = 1_000_000
    for n in range(total_trials):
        outcome = get_ith_outcome(p1,p2)
        distribution[outcome] = distribution.get(outcome,0) + 1/total_trials 
    return distribution


In [4]:
monte_carlo(0.5,0.3)

{(8, 11): 0.1281740000001075,
 (7, 11): 0.09914500000007849,
 (6, 11): 0.10127600000008062,
 (11, 6): 0.018563999999997912,
 (9, 11): 0.10731000000008666,
 (5, 11): 0.12753400000010687,
 (4, 11): 0.09826300000007761,
 (10, 10): 0.11763700000009698,
 (1, 11): 0.01488099999999552,
 (11, 7): 0.029929000000009278,
 (11, 4): 0.0046860000000002976,
 (2, 11): 0.030389000000009738,
 (3, 11): 0.0503510000000297,
 (11, 8): 0.026136000000005485,
 (11, 5): 0.00764400000000069,
 (11, 9): 0.03100100000001035,
 (0, 11): 0.0026140000000000226,
 (11, 3): 0.0032330000000001047,
 (11, 2): 0.0010000000000000152,
 (11, 1): 0.00019399999999999957,
 (11, 0): 3.8999999999999986e-05}

## Дерево

Щоб знайти ймовірності потрапляння в фіксований лист дерева(кінеций результат матчу), нам потрібно знайти всі шляхи від кореня дерева до фіксованого листа дерева, знайти ймовірність пройти по кожному шляху і додати їх.

Я зробив це наступним чином:

Ітерюємося по вершинам (беремо дві дочірні вершини для поточної,якщо вона не кінцева), після одної ітерації записуємо ймовірність потрапити у вершину,в яку ми потрапили, якщо ця вершина - лист, то додаємо цю ймовірність в словник answer 

In [5]:
#Tree 
def Tree(p1,p2):
    answer = dict()
    
    def Tree_Iterate(p1,p2, start_node = (0,0),first_player_serve=True,p=1):
        if start_node[0] == 11 or start_node[1] == 11 or (start_node[0]==10 and start_node[1] == 10):
            answer[start_node] = answer.get(start_node,0) + p
            return None

    
        if check_first_player_serve(start_node[0]+start_node[1]):
            return ((Tree_Iterate(p1,p2,(start_node[0]+1,start_node[1]),p=p*p1)),\
                    (Tree_Iterate(p1,p2,(start_node[0],start_node[1]+1),p=p*(1-p1))))
        else:
            return ((Tree_Iterate(p1,p2,(start_node[0]+1,start_node[1]),p=p*p2)),\
                    (Tree_Iterate(p1,p2,(start_node[0],start_node[1]+1),p=p*(1-p2))))
    
    Tree_Iterate(p1,p2)
    return answer
    

In [6]:
Tree(0.5,0.3)

{(11, 0): 3.796875e-05,
 (11, 1): 0.00020123437499999997,
 (11, 2): 0.0010289531249999993,
 (11, 3): 0.0032883046874999863,
 (11, 4): 0.004799344921874951,
 (11, 5): 0.007638398085937394,
 (11, 6): 0.018604795781249732,
 (11, 7): 0.029976933046888327,
 (11, 8): 0.025919355509773086,
 (11, 9): 0.030780258432996168,
 (10, 10): 0.11776491293460073,
 (9, 11): 0.10720338982348102,
 (8, 11): 0.1280117552519063,
 (7, 11): 0.09888697054685476,
 (6, 11): 0.10129277703124871,
 (5, 11): 0.12810751214844449,
 (4, 11): 0.09787411210937413,
 (3, 11): 0.050487992187499976,
 (2, 11): 0.030500203125000014,
 (1, 11): 0.014968734374999992,
 (0, 11): 0.002626093749999999}

## Процес Маркова

В цьому завданні нам потрібно побудувати матрицю переходів, після чого потрібно знайти стаціонарний вектор, тобто такий вектор p, що p = Transitition_Matrix * p.

Знайдемо всі доступні стани, і створимо матрицю переходів.

In [7]:
available_pos = list()
for i in range(0,12):
    for j in range(0,12):
        if not ((i >= 10 and j > 10) or (i > 10 and j >= 10)):
            available_pos.append((i,j))

In [8]:
print(len(available_pos))
print(available_pos)

141
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (10, 0), (10, 1

З цих станів потрібно зробити матрицю переходу, наприклад з стану (0,0) в стан (1,0) ми будемо переходити з ймовірністю p1, а з (0,0) -> (0,1) з ймовірністю 1-p1. Матриця матиме розмірність 141х141, стовпчик вказуватиме в якому стані ми є зараз, а рядок в який переходимо.

Для зручності відсортуємо стани  по сумі очок, та представимо таблицю у форматі датафрейму.


In [9]:
available_pos.sort(key = lambda x : x[0]+x[1])

In [10]:
print(available_pos)

[(0, 0), (0, 1), (1, 0), (0, 2), (1, 1), (2, 0), (0, 3), (1, 2), (2, 1), (3, 0), (0, 4), (1, 3), (2, 2), (3, 1), (4, 0), (0, 5), (1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (0, 6), (1, 5), (2, 4), (3, 3), (4, 2), (5, 1), (6, 0), (0, 7), (1, 6), (2, 5), (3, 4), (4, 3), (5, 2), (6, 1), (7, 0), (0, 8), (1, 7), (2, 6), (3, 5), (4, 4), (5, 3), (6, 2), (7, 1), (8, 0), (0, 9), (1, 8), (2, 7), (3, 6), (4, 5), (5, 4), (6, 3), (7, 2), (8, 1), (9, 0), (0, 10), (1, 9), (2, 8), (3, 7), (4, 6), (5, 5), (6, 4), (7, 3), (8, 2), (9, 1), (10, 0), (0, 11), (1, 10), (2, 9), (3, 8), (4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (9, 2), (10, 1), (11, 0), (1, 11), (2, 10), (3, 9), (4, 8), (5, 7), (6, 6), (7, 5), (8, 4), (9, 3), (10, 2), (11, 1), (2, 11), (3, 10), (4, 9), (5, 8), (6, 7), (7, 6), (8, 5), (9, 4), (10, 3), (11, 2), (3, 11), (4, 10), (5, 9), (6, 8), (7, 7), (8, 6), (9, 5), (10, 4), (11, 3), (4, 11), (5, 10), (6, 9), (7, 8), (8, 7), (9, 6), (10, 5), (11, 4), (5, 11), (6, 10), (7, 9), (8, 8), (9, 7), (10, 6)

In [11]:
table = pd.DataFrame(np.zeros((141,141)),index=available_pos,columns=available_pos)
table

,"(0, 0)","(0, 1)","(1, 0)","(0, 2)","(1, 1)","(2, 0)","(0, 3)","(1, 2)","(2, 1)","(3, 0)",...,"(9, 9)","(10, 8)","(11, 7)","(8, 11)","(9, 10)","(10, 9)","(11, 8)","(9, 11)","(10, 10)","(11, 9)"
"(0, 0)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(1, 0)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(1, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(10, 9)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(11, 8)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(9, 11)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(10, 10)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Використаємо функцію fill_col, для того, щоб коректно записати ймовірності в матрицю переходів.

In [12]:
def fill_col(idx_from,p1,p2):
    col = table[idx_from]
    columns = table.columns
    first_player_score = idx_from[0] 
    second_player_score = idx_from[1]

    #Розглядаємо переходи з кінцевого стану
    if first_player_score == 11 or second_player_score == 11 or \
            (first_player_score == second_player_score and second_player_score == 10):
        col[idx_from] = 1
        return col
    #Розгядаємо переходи не з кінцевих станів
    if check_first_player_serve(first_player_score+second_player_score):
            col[(first_player_score+1,second_player_score)] = p1
            col[(first_player_score,second_player_score+1)] = 1-p1
    else:
            col[(first_player_score+1,second_player_score)] = p2
            col[(first_player_score,second_player_score+1)] = 1-p2
    return col
                
            
    

In [13]:
for idx_from in table.index:
    table[idx_from] = fill_col(idx_from,0.5,0.3)
    

p = np.zeros(141)
p[0] = 1
p = np.dot(np.linalg.matrix_power(table,20),p)    
    
states = table.columns    
answer = dict()

for state,probability in zip(states,p):
    if probability > 0:
        answer[state] = probability
answer


{(0, 11): 0.002626093749999999,
 (11, 0): 3.796875e-05,
 (1, 11): 0.014968734374999994,
 (11, 1): 0.000201234375,
 (2, 11): 0.030500203124999993,
 (11, 2): 0.001028953125,
 (3, 11): 0.05048799218749999,
 (11, 3): 0.0032883046874999998,
 (4, 11): 0.09787411210937497,
 (11, 4): 0.004799344921875,
 (5, 11): 0.12810751214843746,
 (11, 5): 0.007638398085937499,
 (6, 11): 0.10129277703124996,
 (11, 6): 0.018604795781249996,
 (7, 11): 0.09888697054687497,
 (11, 7): 0.02997693304687499,
 (8, 11): 0.12801175525195307,
 (11, 8): 0.025919355509765617,
 (9, 11): 0.10720338982363277,
 (10, 10): 0.11776491293476557,
 (11, 9): 0.0307802584330078}

In [14]:
# Матриця переходів, рядок вказує на поточний стан, стовпчик на наступний.
table.T

,"(0, 0)","(0, 1)","(1, 0)","(0, 2)","(1, 1)","(2, 0)","(0, 3)","(1, 2)","(2, 1)","(3, 0)",...,"(9, 9)","(10, 8)","(11, 7)","(8, 11)","(9, 10)","(10, 9)","(11, 8)","(9, 11)","(10, 10)","(11, 9)"
"(0, 0)",0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, 1)",0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(1, 0)",0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(1, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(10, 9)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.3
"(11, 8)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(9, 11)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(10, 10)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# 2 

## Монте-Карло

Монте-Карло в цьому варіанті не сильно відрізняється, єдиною відмінністю є те, що тут ми закінчуємо матч (знаходимо переможця), коли модуль різниці очок = 2.

In [15]:
def monte_carlo2(p1):
    first_player_won_probability = 0 
    second_player_won_probability = 0 
    total_trials = 1_000_000
    for n in range(total_trials):
        first_player_serve = True
        score_diff = 0
        while abs(score_diff) != 2:           
            if np.random.uniform() < p1:
                score_diff += 1
            else:
                score_diff -= 1
        
        if score_diff == 2:
            first_player_won_probability += 1/total_trials
        else:
            second_player_won_probability += 1/total_trials
    return {'first_player_won_probability':first_player_won_probability,
            "second_player_won_probability":second_player_won_probability}
            

In [16]:
monte_carlo2(0.3)

{'first_player_won_probability': 0.15569100000013503,
 'second_player_won_probability': 0.8443090000034412}

## Дерево

Оскільки існує ненульова ймовірність зіграти n подач для будь-якого натурального n, то тут ми повинні деяким чином обрізати  дерево, я вирішив обрізати тоді, коли ймовірність потрапити в деякий стан буде менша за фіксований епсілон.

In [17]:
def Tree2(p):
    answer = dict()
    
    def Tree_Iterate(p1, start_node = 0,p=1,eps = 1e-10):
        if start_node == 2 or start_node == -2 :
            answer[start_node] = answer.get(start_node,0) + p
            return None
        if p*p1 > eps or p*(1-p1) > eps:
            return ((Tree_Iterate(p1,start_node+1,p=p*p1)),\
                    (Tree_Iterate(p1,start_node-1,p=p*(1-p1))))
       
    Tree_Iterate(p)
    return {'first_player_won_probability':answer[2],'second_player_won_probability':answer[-2]}

Tree2(0.3)

{'first_player_won_probability': 0.1551715890768142,
 'second_player_won_probability': 0.8448257003555246}

## Процес Маркова

Тут матриця переходів, дещо інша рядок вказує на стан, в якому ми зараз, а стовпчик на стан, в який перейдемо. 
Розглядаємо наступні стани {-2,-1,0,1,2}, стан -2 -- 2 очка в користь першого гравця, 2 -- 2 очка в корисить другого гравця.

In [18]:
def get_matrix(p):
    return np.array([[1,0,0,0,0],
                     [p,0,1-p,0,0],
                     [0,p,0,1-p,0],
                     [0,0,p,0,1-p],
                     [0,0,0,0,1]])
Transitiotion_matrix = get_matrix(0.3)

In [19]:
Transitiotion_matrix

array([[1. , 0. , 0. , 0. , 0. ],
       [0.3, 0. , 0.7, 0. , 0. ],
       [0. , 0.3, 0. , 0.7, 0. ],
       [0. , 0. , 0.3, 0. , 0.7],
       [0. , 0. , 0. , 0. , 1. ]])

In [20]:
p = np.array([0,0,1,0,0])
Z = Transitiotion_matrix.copy()
p = np.dot(p,np.linalg.matrix_power(Transitiotion_matrix,200))
answer = {'first_player_won_probability':p[0],'second_player_won_probability':p[4]}

In [21]:
answer

{'first_player_won_probability': 0.15517241379310345,
 'second_player_won_probability': 0.8448275862068964}

# 3

Порівняємо спочатку методи з першого завдання, а потім з другого.
### Методи першого завдання
Найповільнішим методом є Монте-Карло через те, що нам потрібно моделювати велику кількість матчів, а якщо цього не зробити то ми отримаємо меншу точність. До того ж, він є найменш точним, оскільки є випадковим. В цьому методі ми балансуємо між швидкістю/точністю. (більше ітерацій = менша швидкість => більша точність, менше ітерацій = більша швидкість => менша точність). Перевагою Монте-Карло є його простота(інтуїтивність).

Щодо другого та третього методу, вони є однаковими по точності, але третій дещо швидший за другий. 
Суб'єктивним недоліком третього методу є необхідність підготувати матрицю переходів, що робить його більш громіздким.

В результаті маємо наступне:

Монте-Карло - повільний, непогана точність, компактність, інтуїтивність/простий для розуміння, accuracy-speed tradeoff. 

Дерева - швидкий, точний, компактний, достатньо простий для розуміння.

Процес Маркова - швидкий, точний, більш громіздкий, не дуже інтуїтивний.

### Методи другого завдання
В другому завданні ситуація дещо змінилася,тому напишу лише зміни, які відбулися.
Метод Монте-Карло залишає собі свої недоліки та переваги.

Дерева стали менш точними, оскільки розглянути всі можливі стани неможливо.

Процес Маркова теж втратив дещо в точності в моїй реалізації, щодо громіздкості, то в цьому завдані матриця переходів проста, тому метод став досить компактним.







